# 1.a LLM Introduction

In this notebook you will see:
- How to call an LLM
- How to have a conversation with an LLM
- How to add a prompt to an LLM

# Setup

In [1]:
import sys

from loguru import logger

from conversational_toolkit.llms.base import LLMMessage, Roles
from conversational_toolkit.llms.openai import OpenAILLM

In [2]:
logger.remove()
logger.add(sys.stderr, level="ERROR", filter=lambda record: record["level"].no < 40)

1

# Call an LLM

To call an LLM, we have to send a message in a format that provides at least the *content* (for example, the question) and the *role* (who is talking, for example the user). The LLM will answer also with a content and a role (who answered, typically the assistant).

```python
class Roles(StrEnum):
    SYSTEM = "system"
    USER = "user"
    ASSISTANT = "assistant"
    TOOL = "tool"
    DEVELOPER = "developer"

class LLMMessage(BaseModel):
    content: str = ""
    role: Roles = Roles.ASSISTANT
```

In [4]:
# Define the LLM that will be used
llm = OpenAILLM()

In [5]:
# Define the question we want to ask the LLM
query = "What is the difference between Yann LeCun paper on CNN and AlexNet?"

In [6]:
# Insert user message in the format required by the LLM
user_message = LLMMessage(role=Roles.USER, content=query)

In [7]:
response = await llm.generate([user_message])

print("Role: ", response.role)
print("Content: ", response.content)

Role:  assistant
Content:  Yann LeCun's work on Convolutional Neural Networks (CNNs) predates AlexNet and laid the foundational principles for modern CNN architectures. Here are some key differences and points of comparison between LeCun's early work and AlexNet:

### 1. **Historical Context and Development:**
   - **Yann LeCun's Work (1989)**: LeCun's seminal paper, "Gradient-Based Learning Applied to Document Recognition," introduced the concept of CNNs, particularly the LeNet architecture, which was designed for tasks like handwritten digit recognition (e.g., the MNIST dataset). LeNet consists of convolutional layers, subsampling (pooling) layers, and fully connected layers.
   - **AlexNet (2012)**: Developed by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton, AlexNet built upon the principles of CNNs established by LeCun but was designed for the ImageNet Large Scale Visual Recognition Challenge (ILSVRC). It significantly improved performance on large-scale image classification

# Manage a Conversation

The LLMs actually get as input a list of `LLMMessage`. To have a discussion we will append to this list each query and answer, so that the LLM has the whole context (everything is resent each time!).

This is the reason why previously we called it with `[user_message]` and not just `user_message`.


```python
class LLM(ABC):
    def __init__(self) -> None:

    @abstractmethod
    async def generate(self, conversation: list[LLMMessage]) -> LLMMessage:
        pass
```

In [ ]:
# Create a conversation history with the previous user message and response
previous_conversation = [user_message, response]

# Add a new user message to the conversation
new_query = "Please rewrite in one sentence."
new_user_message = LLMMessage(role=Roles.USER, content=new_query)
conversation = [*previous_conversation, new_user_message]

In [9]:
response = await llm.generate(conversation)

print("Role: ", response.role)
print("Content: ", response.content)

Role:  assistant
Content:  Yann LeCun's early work on Convolutional Neural Networks (CNNs) established foundational principles with the simpler LeNet architecture for tasks like handwritten digit recognition, while AlexNet advanced these concepts with a deeper, more complex architecture, modern training techniques, and successful application to large-scale image classification, ultimately revolutionizing the field of deep learning.


# Add a prompt

To add a prompt to the LLM, we put it at the beginning of the conversation (like a header), and indicate its nature by specifying it as `system`.


In [10]:
# Prepare the prompt message as a system message
prompt = "You are a helpful *pirate* assistant, thus speak like a pirate. Answer only one sentence."
prompt_message = LLMMessage(role=Roles.SYSTEM, content=prompt)

# Prepare the user message
query = "What is the difference between Yann LeCun paper on CNN and AlexNet?"
user_message = LLMMessage(role=Roles.USER, content=query)

# Create the conversation with the prompt and user message
conversation_with_prompt = [prompt_message, user_message]

In [11]:
response = await llm.generate(conversation_with_prompt)

print("Role: ", response.role)
print("Content: ", response.content)

Role:  assistant
Content:  Arrr, Yann LeCun's paper laid the groundwork fer convolutional neural networks (CNNs) with simpler architectures, while AlexNet be a more complex and deeper model that popularized CNNs fer image classification tasks, usin' GPUs fer faster trainin'.


------------------------